## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## Custom modules
import sys
import os
os.environ['PATH'] = os.environ['PATH'] + ':/Library/TeX/texbin'
sys.path.append('/Users/riccardo/Documents/GitHub/ImmuneResponseCOVID19/')
from Modules import Parameters, DataPreprocessing as DP, Classification as CL, CustomFunctions as CF

## Parameters

In [ ]:
## Multivariate sets
FC_set = Parameters.FC_set
Dem_set = Parameters.Dem_set
CK_set = Parameters.CK_set
BM_set =  Parameters.BM_set

## Targets
target_train = Parameters.train_target
target_test = Parameters.test_target

## Age
lower_bound = Parameters.age_min
upper_bound = Parameters.age_max

## Delta onset
lower_bound_donset = Parameters.donset_min
upper_bound_donset = Parameters.donset_max

## Minimum NPV
min_NPV_Models = Parameters.min_NPV_Models
min_NPV = Parameters.min_NPV

## Nan masking row-wise
do_nan_masking = Parameters.do_nan_masking
do_nan_masking_univ = Parameters.do_nan_masking_univ
nan_masking = Parameters.nan_masking
do_nan_masking_groupwise = Parameters.do_nan_masking_groupwise

## N samples for average
N_av = Parameters.N_av

## Imputation
imputation_method = Parameters.imputation_method

## Standardization
std_method = Parameters.std_method
std_cat_variables = Parameters.std_cat_variables

## PCA % var. threshold
pc_var_th = Parameters.pca_var_threshold

## Preprocessing
do_preprocessing = Parameters.do_preprocessing_multiv

## Train-test
test_size = Parameters.test_size
ignore_sex = Parameters.ignore_sex

## Paths
path_results = Parameters.path_results
path_figures = Parameters.path_figures
path_setsdescription = Parameters.path_setsdescription
exp_description = Parameters.exp_description
exp_multiv_description = Parameters.exp_multiv_description
foldername_multiv = Parameters.foldername_multiv

## Run experiments
run_experiments = True

In [ ]:
print('Age min:', lower_bound)
print('Age max:', upper_bound)
print('Donset min:', lower_bound_donset)
print('Donset max:', upper_bound_donset)

## Import and prepare datasets

In [ ]:
DataInpatients, DataOutpatients = DP.data_preprocessing()

# Mask data by target
mask = pd.notnull(DataInpatients[target_train].values) & pd.notnull(DataInpatients[target_test].values)
DataInpatients = DataInpatients.loc[mask,:]

DataInpatients.shape

## Models dictionary

In [ ]:
models_dict = {}

## LR models
models_dict['LR'] = {}
models_dict['LR']['FC'] = FC_set
models_dict['LR']['FC+Dem'] = FC_set + Dem_set
models_dict['LR']['CK'] = CK_set
models_dict['LR']['CK+Dem'] = CK_set + Dem_set
models_dict['LR']['BM'] = BM_set
models_dict['LR']['BM+Dem'] = BM_set + Dem_set

## Datasets description

In [ ]:
nan_masking_val = 0.5
do_nan_masking_groupwise_val = True
groups = [FC_set, CK_set, Dem_set, BM_set]

CCI_th = 2
if lower_bound>60:
    CCI_th = 4
dict_stats = { 'N': [], 'NANs': [], 'sex': [], 'age': [], 'd. onset': [], 'CCI<%d'%CCI_th: [], 'OTI+death': []}

for model_name in models_dict['LR'].keys():
    print(model_name)
    
    # Mask data
    Data = DataInpatients.copy()
    _, row_mask = CL.nan_masking_fc(Data, models_dict['LR'][model_name], 
                                    nan_masking=nan_masking_val, 
                                    do_nan_masking_groupwise=do_nan_masking_groupwise_val, 
                                    groups=groups, 
                                    return_row_mask=True)

    # N. patients
    n_patients = sum(row_mask)
    val_str = '%d' % n_patients
    #print('N: '+val_str)
    dict_stats['N'].append(val_str)

    # NANs
    n = DataInpatients.loc[row_mask, models_dict['LR'][model_name]].size
    n_nan = sum(DataInpatients.loc[row_mask, models_dict['LR'][model_name]].isna().values.flatten())
    ratio = n_nan/n*100
    val_str = '%.1f%%' % ratio
    #print('NANs: '+val_str)
    dict_stats['NANs'].append(val_str)
    
    # Sex
    n = sum(DataInpatients.loc[row_mask, 'sex'] == 1)
    ratio = n/n_patients*100
    val_str = '%d(%.1f%%)' %(n, ratio)
    #print('sex: '+val_str)
    dict_stats['sex'].append(val_str)

    # Age
    q1, median, q3 = DataInpatients.loc[row_mask, 'age'].quantile([0.25,0.5,0.75])
    val_str = '%d(%d-%d)' % (round(median), round(q1), round(q3))
    #print('age: '+val_str)
    dict_stats['age'].append(val_str)

    # D. onset
    q1, median, q3 = DataInpatients.loc[row_mask, 'delta_onset'].quantile([0.25,0.5,0.75])
    val_str = '%d(%d-%d)' % (round(median), round(q1), round(q3))
    #print('d. onset: '+val_str)
    dict_stats['d. onset'].append(val_str)

    # CCI
    n = sum(DataInpatients.loc[row_mask, 'CCI (charlson comorbidity index)'] < CCI_th)
    ratio = n/n_patients*100
    val_str = '%d(%.1f%%)' %(n, ratio)
    #print('CCI<2: '+val_str)
    dict_stats['CCI<%d'%CCI_th].append(val_str)

    # Outcome
    n = sum(DataInpatients.loc[row_mask, 'OTI+death'] == 1)
    ratio = n/n_patients*100
    val_str = '%d(%.1f%%)' %(n, ratio)
    #print('CCI<2: '+val_str)
    dict_stats['OTI+death'].append(val_str)

    #print('\n')

df_stat = pd.DataFrame(dict_stats, index=models_dict['LR'].keys())

# Export table
filename = 'DatasetsDescription'+exp_description+'.xlsx'
df_stat.to_excel(path_setsdescription+filename)

df_stat


## Run analysis - multiple splits

In [ ]:
if run_experiments:
    Data = DataInpatients.copy()     
    
    ## Initialize results dictionaries
    Results_N_av = {}
    Ground_Truth_N_av = {}
    for model in models_dict.keys():
        Results_N_av[model] = {}
        Ground_Truth_N_av[model] = {}
        for set_name in models_dict[model].keys():
            Results_N_av[model][set_name] = {'Train':[], 'Test':[], 'Train_value': [], 'Test_value': [], 'Weights':[], 'Bias':[], 'C':[], 'Std_Parameters': []}
            Ground_Truth_N_av[model][set_name] = {'Train':[], 'Test':[]}
            if min_NPV_Models:
                Results_N_av[model][set_name+'_minNPV'] = {'Train':[], 'Test':[], 'Train_value': [], 'Test_value': []}
                Ground_Truth_N_av[model][set_name+'_minNPV'] = {'Train':[], 'Test':[]}


    ## Sets of variables for nan removal
    groups = [FC_set, CK_set, Dem_set, BM_set]


    ## Run average
    fix_outliers = False
    do_imputation = True
    print('Running average...')
    print('Fix outliers: %s' % fix_outliers)
    print('Do imputation: %s' % do_imputation)
    print('Nan masking groupwise: %s' % do_nan_masking_groupwise)
    print('Do preprocessing: %s' % do_preprocessing)
    for i_split in range(N_av): 
        idx_split = i_split+1
        print('%d/%d' %(idx_split, N_av))
        
        Results = CL.models_prediction(Data=Data.copy(),
                                       test_size=test_size,
                                       models_dict=models_dict,
                                       target_train=target_train, 
                                       target_test=target_test, 
                                       min_NPV=min_NPV,
                                       min_NPV_Models=min_NPV_Models,
                                       standardization=std_method, 
                                       imputation=imputation_method, 
                                       do_nan_masking=do_nan_masking,
                                       do_nan_masking_univ=do_nan_masking_univ,
                                       nan_masking=nan_masking, 
                                       do_nan_masking_groupwise=do_nan_masking_groupwise, 
                                       groups=groups,
                                       do_preprocessing=do_preprocessing, 
                                       fix_outliers=fix_outliers,
                                       do_imputation=do_imputation, 
                                       ignore_sex=ignore_sex, 
                                       std_cat_variables=std_cat_variables,
                                       random_state=idx_split)

        for model in Results_N_av.keys():
            for set_name in Results_N_av[model].keys():
                Ground_Truth_N_av[model][set_name]['Train'].append(Results[model][set_name]['Train_Labels'])
                Ground_Truth_N_av[model][set_name]['Test'].append(Results[model][set_name]['Test_Labels'])
                Results_N_av[model][set_name]['Train_value'].append(Results[model][set_name]['Train_value'])
                Results_N_av[model][set_name]['Test_value'].append(Results[model][set_name]['Test_value'])
                Results_N_av[model][set_name]['Train'].append(Results[model][set_name]['Train'])
                Results_N_av[model][set_name]['Test'].append(Results[model][set_name]['Test'])
                
                if model=='LR' and '_minNPV' not in set_name:
                    Results_N_av[model][set_name]['Weights'].append(Results[model][set_name]['Weights'])
                    Results_N_av[model][set_name]['Bias'].append(Results[model][set_name]['Bias'])
                    Results_N_av[model][set_name]['C'].append(Results[model][set_name]['C'])
                    Results_N_av[model][set_name]['Std_Parameters'].append(Results[model][set_name]['Std_Parameters'])

### Parameters dictionary

In [ ]:
if run_experiments:
    parameters_dict = {}
    std_parameters_dict = {}

    if 'LR' in Results_N_av.keys():
        parameters_dict['LR'] = {}
        std_parameters_dict['LR'] = {}

        for set_name in Results_N_av['LR'].keys():
            if '_minNPV' not in set_name:
                weights = np.array(Results_N_av['LR'][set_name]['Weights'])
                bias = np.array(Results_N_av['LR'][set_name]['Bias'])
                C_reg = np.array(Results_N_av['LR'][set_name]['C'])
                Std_Parameters = np.array(Results_N_av['LR'][set_name]['Std_Parameters'])

                if weights.shape[1]>1:
                    column_names = models_dict['LR'][set_name]
                    if '#Dis' in set_name:
                        new_columns = []
                        bool_idx = []
                        for col in column_names:
                            v = [col1 for col1 in Data.columns if col+'_INT' in col1]
                            new_columns += v
                            if len(v)>0:
                                bool_idx += [False]
                            else:
                                bool_idx += [True]
                        new_columns += list(np.array(column_names)[bool_idx])
                        column_names = new_columns
                    df = pd.DataFrame(weights, columns=column_names)
                    df.insert(loc=0, column='bias', value=bias)
                    parameters_dict['LR'][set_name] = df

                    column_names = models_dict['LR'][set_name]
                    if '#Dis' not in set_name:
                        df = pd.DataFrame(Std_Parameters, columns=[col for col in column_names if col!='sex'])
                        df.insert(loc=0, column='C', value=C_reg)
                    else:
                        df = pd.DataFrame(C_reg, columns=['C'])
                    std_parameters_dict['LR'][set_name] = df

### Compute performanes

In [ ]:
if run_experiments:

    Performances = {}

    Scores = {'roc_auc_score': roc_auc_score,
              'f1': f1_score,
              'recall': recall_score,
              'precision': precision_score,
              'NPV': precision_score,
              'specificity': recall_score,
              'accuracy': accuracy_score}

    for model in Results_N_av.keys():
        Performances[model] = {}

        for set_name in Results_N_av[model].keys():
            #print(model, set_name)
            Performances[model][set_name] = {}
            print(model, set_name)

            for score in Scores.keys():
                Performances[model][set_name][score] = {'Train': [], 'Test': []}
                sc = Scores[score]
                for sample in range(N_av):
                    y_pred_train = Results_N_av[model][set_name]['Train'][sample]
                    y_value_train = Results_N_av[model][set_name]['Train_value'][sample]
                    y_train = Ground_Truth_N_av[model][set_name]['Train'][sample]
                    y_pred_test = Results_N_av[model][set_name]['Test'][sample]
                    y_value_test = Results_N_av[model][set_name]['Test_value'][sample]
                    y_test = Ground_Truth_N_av[model][set_name]['Test'][sample]

                    y_pred_train_1 = 1 - y_pred_train
                    y_train_1 = 1 - y_train
                    y_pred_test_1 = 1 - y_pred_test
                    y_test_1 = 1 - y_test

                    if score in ['NPV', 'specificity']:
                        if sum(pd.notnull(y_pred_train_1))>1:
                            score_val_train = sc(y_train_1, y_pred_train_1)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test_1))>1:
                            score_val_test = sc(y_test_1, y_pred_test_1)
                        else:
                            score_val_test = np.nan
                    elif score=='roc_auc_score':
                        if sum(pd.notnull(y_pred_train))>1:
                            score_val_train = sc(y_train, y_value_train)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test))>1:
                            score_val_test = sc(y_test, y_value_test)
                        else:
                            score_val_test = np.nan
                    else:
                        if sum(pd.notnull(y_pred_train))>1:
                            score_val_train = sc(y_train, y_pred_train)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test_1))>1:
                            score_val_test = sc(y_test, y_pred_test)
                        else:
                            score_val_test = np.nan

                    Performances[model][set_name][score]['Train'].append(score_val_train)
                    Performances[model][set_name][score]['Test'].append(score_val_test)

#### Average Performances

In [ ]:
if run_experiments:

    # Train performances
    test_or_train = 'Train'
    Performances_mean_ci_train = {}
    Performances_mean_ci_train_0 = {}
    notnull_prop = 0.9
    normality_pval_th = 0.001

    for score in Scores:
        Performances_mean_ci_train[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}
        Performances_mean_ci_train_0[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}

    for model in Performances.keys():
        for set_name in Performances[model].keys():
            for score in Performances[model][set_name].keys():

                v = np.array(Performances[model][set_name][score][test_or_train])
                mask = pd.notnull(v)
                if sum(mask)>notnull_prop*len(mask):
                    v = v[pd.notnull(v)]
                    # Normality test
                    normality_pval = st.normaltest(v, nan_policy='omit')[1]
                    if normality_pval>normality_pval_th:
                        ci_lower, ci_upper = st.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=st.sem(v))
                        score_mean = np.mean(v)
                    else:
                        ci_lower, ci_upper, score_mean = CF.bootstrap_ci_mean(v)
                else:
                    ci_upper = 0
                    ci_lower = 0 
                    score_mean = 0

                if '_minNPV' not in set_name:
                    Performances_mean_ci_train[score]['Mean'].append(score_mean)
                    Performances_mean_ci_train[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_train[score]['CI_upper'].append(ci_upper)
                else:
                    Performances_mean_ci_train_0[score]['Mean'].append(score_mean)
                    Performances_mean_ci_train_0[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_train_0[score]['CI_upper'].append(ci_upper)


    # Test performances
    test_or_train = 'Test'
    Performances_mean_ci_test = {}
    Performances_mean_ci_test_0 = {}

    for score in Scores:
        Performances_mean_ci_test[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}
        Performances_mean_ci_test_0[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}

    for model in Performances.keys():
        for set_name in Performances[model].keys():
            for score in Performances[model][set_name].keys():

                v = np.array(Performances[model][set_name][score][test_or_train])
                mask = pd.notnull(v)
                if sum(mask)>notnull_prop*len(mask):
                    v = v[pd.notnull(v)]
                    # Normality test
                    normality_pval = st.normaltest(v, nan_policy='omit')[1]
                    if normality_pval>normality_pval_th:
                        ci_lower, ci_upper = st.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=st.sem(v))
                        score_mean = np.mean(v)
                    else:
                        ci_lower, ci_upper, score_mean = CF.bootstrap_ci_mean(v)
                else:
                    ci_upper = 0
                    ci_lower = 0 
                    score_mean = 0

                if '_minNPV' not in set_name:
                    Performances_mean_ci_test[score]['Mean'].append(score_mean)
                    Performances_mean_ci_test[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_test[score]['CI_upper'].append(ci_upper)
                else:
                    Performances_mean_ci_test_0[score]['Mean'].append(score_mean)
                    Performances_mean_ci_test_0[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_test_0[score]['CI_upper'].append(ci_upper)

### Export results

In [ ]:
if run_experiments:
    # Create directory
    try:
        os.mkdir(path_results+foldername_multiv)
    except OSError:
        print ("Creation of the directory failed")
    else:
        print ("Successfully created the directory")

    path_export = path_results+foldername_multiv

    # Export 

    # Tuples here are: (name_score, mean) (name_score, CI_lower) (name_score, CI_upper) #
    my_tuples = [(score, val) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]
    new_columns = pd.MultiIndex.from_tuples(my_tuples)
    index = []
    for model in models_dict.keys():
        index.extend([model+': '+name for name in models_dict[model].keys()])
    n_models = len(index)
    values = np.array([np.array(Performances_mean_ci_train[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
    df_results = pd.DataFrame(values, columns=new_columns, index=index)
    filename = 'performances_train.xlsx'
    df_results.to_excel(path_export+filename, engine='xlsxwriter')

    if min_NPV_Models:
        values_0 = np.array([np.array(Performances_mean_ci_train_0[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
        df_results_0 = pd.DataFrame(values_0, columns=new_columns, index=index)
        filename_0 = 'performances_minNPV_train.xlsx'
        df_results_0.to_excel(path_export+filename_0, engine='xlsxwriter')


    # Tuples here are: (name_score, mean) (name_score, CI_lower) (name_score, CI_upper) #
    values = np.array([np.array(Performances_mean_ci_test[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
    df_results = pd.DataFrame(values, columns=new_columns, index=index)
    filename = 'performances_test.xlsx'
    df_results.to_excel(path_export+filename, engine='xlsxwriter')

    if min_NPV_Models:
        values_0 = np.array([np.array(Performances_mean_ci_test_0[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
        df_results_0 = pd.DataFrame(values_0, columns=new_columns, index=index)
        filename_0 = 'performances_minNPV_test.xlsx'
        df_results_0.to_excel(path_export+filename_0, engine='xlsxwriter')

    # Export models description
    models_description = {}
    for model in models_dict.keys():
        for name in models_dict[model].keys():
            description = ' # '.join(models_dict[model][name])
            models_description[model+'#'+name] = description

    filename = 'models_description.xlsx'
    pd.Series(models_description).to_excel(path_export+filename)


    # Export models parameters
    for set_name in parameters_dict['LR'].keys():
        df = parameters_dict['LR'][set_name]
        filename = 'Parameters_model#%s_name#%s.xlsx' % ('LR', set_name)
        df.to_excel(path_export+filename)


    # Export models std parameters
    for set_name in std_parameters_dict['LR'].keys():
        df = std_parameters_dict['LR'][set_name]
        filename = 'StdParameters_model#%s_name#%s.xlsx' % ('LR', set_name)
        df.to_excel(path_export+filename)

## Figures

In [ ]:
# Color scheme
color_0 = Parameters.dark_blue
color_1 = Parameters.red_purp
color_control = Parameters.green

# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5

### 2D Plot - test outpatients

In [ ]:
model_name = 'CK+Dem' 

Data_train = DataInpatients.copy()
Data_test = DataOutpatients.copy()
if 'FC' not in model_name:
    Data_test = DataInpatients.copy() 
    
print(Data_train.shape, Data_test.shape)

In [ ]:
random_state = None
columns = models_dict['LR'][model_name]
fix_outliers = False
do_imputation = True
groups = [FC_set, CK_set, Dem_set, BM_set]

results, MLR, prep_data = CL.LR_model_results(Data=Data_train,
                                              Data_test=Data_test,
                                              features=columns, 
                                              set_name=model_name, 
                                              target_train=target_train, 
                                              target_test=target_test, 
                                              test_size=test_size,
                                              hyperp_dict={}, 
                                              do_preprocessing=do_preprocessing, 
                                              fix_outliers=fix_outliers, 
                                              do_imputation=do_imputation, 
                                              imputation=imputation_method, 
                                              pca_var_threshold=pc_var_th, 
                                              standardization=std_method, 
                                              min_NPV_Models=min_NPV_Models,
                                              min_NPV=min_NPV, 
                                              groups=groups, 
                                              do_nan_masking=do_nan_masking, 
                                              do_nan_masking_groupwise=do_nan_masking_groupwise, 
                                              do_nan_masking_univ=do_nan_masking_univ, 
                                              nan_masking=nan_masking,
                                              return_model=True, 
                                              return_data=True)

In [ ]:
# Prepare data for 2D plot
Data_X = prep_data['Train']
Data_y = results[model_name]['Train_Labels']
Data_X_t = prep_data['Test']
proj_2D = CF.MLR_axis(MLR, Data_X, Data_test=Data_X_t, use_bias=False) # use_bias=False
X2D_train = proj_2D['Train']
X2D_test = proj_2D['Test']
X1D_mlr_train, X1D_pca_train = X2D_train[:, 0], X2D_train[:, 1]
X1D_mlr_test, X1D_pca_test = X2D_test[:, 0], X2D_test[:, 1]
X_1, X_2 = X1D_mlr_train, X1D_pca_train
X_1_0 = X_1[Data_y==0].copy()
X_2_0 = X_2[Data_y==0].copy()
X_1_1 = X_1[Data_y==1].copy()
X_2_1 = X_2[Data_y==1].copy()
X_1_t, X_2_t = X1D_mlr_test.copy(), X1D_pca_test.copy()

## Define grid
h = .05 # .02  # step size in the mesh
pref_lims = 0.2
offset_lims = 0.
x_1_min, x_1_max = X_1.min() - pref_lims*(X_1.max()-X_1.min()), X_1.max() + pref_lims*(X_1.max()-X_1.min())
x_2_min, x_2_max = X_2.min() - pref_lims*(X_2.max()-X_2.min()), X_2.max() + pref_lims*(X_2.max()-X_2.min())
X_1_t, X_2_t = X1D_mlr_test, X1D_pca_test
x_1_min_t, x_1_max_t = X_1_t.min() - pref_lims*(X_1_t.max()-X_1_t.min()), X_1_t.max() + pref_lims*(X_1_t.max()-X_1_t.min())
x_2_min_t, x_2_max_t = X_2_t.min() - pref_lims*(X_2_t.max()-X_2_t.min()), X_2_t.max() + pref_lims*(X_2_t.max()-X_2_t.min())
if x_1_min_t<x_1_min:
    x_1_min = x_1_min_t
if x_2_min_t<x_2_min:
    x_2_min = x_2_min_t
if x_1_max_t>x_1_max:
    x_1_max = x_1_max_t
if x_2_max_t>x_2_max:
    x_2_max = x_2_max_t  

In [ ]:
## Evaluate predictions
y_train = Data_y.copy()
y_pred = MLR.predict(Data_X) #MLR.fit(Data_X, Data_y).predict(Data_X)
y_pred_t = MLR.predict(Data_X_t)
value_pred = MLR.decision_function(Data_X)
min_NPV_2DPlot = 0.96 # min_NPV
threshold_0 = CL.best_threshold_class0(y_pred=y_pred, value_pred=value_pred, y_target=y_train, min_NPV=min_NPV_2DPlot, fixed_threshold=False)

## Compute scores
f1_train_model = f1_score(y_train, y_pred)
rec_train_model = recall_score(y_train, y_pred)
pre_train_model = precision_score(y_train, y_pred)
NPV_train_model = precision_score(1-y_train, 1-y_pred)
spe_train_model = recall_score(1-y_train, 1-y_pred)
roc_train_model = roc_auc_score(y_train, value_pred)
print('F1:', f1_train_model, 'NPV:', NPV_train_model, 'ROC_AUC:', roc_train_model)

y_pred_0 = np.zeros_like(y_pred)
y_pred_0[value_pred<=threshold_0] = 0
y_pred_0[value_pred>threshold_0] = 1
f1_train_minNPVmodel = f1_score(y_train, y_pred_0)
rec_train_minNPVmodel = recall_score(y_train, y_pred_0)
pre_train_minNPVmodel = precision_score(y_train, y_pred_0)
NPV_train_minNPVmodel = precision_score(1-y_train, 1-y_pred_0)
spe_train_minNPVmodel = recall_score(1-y_train, 1-y_pred_0)
print('F1 (min NPV):', f1_train_minNPVmodel, 'NPV (min NPV):', NPV_train_minNPVmodel)

#### Scatterplot LR score (x) and PC1 (perp to x)

In [ ]:
# Plot parameters
magnification = 2.8
plot_heatmap = False
plot_test = True
plot_immunoparalysis = False
alpha_bg = 1
mec = 'white'
ms = 8
mew = 0 #0.05
c_0 = Parameters.dark_blue
c_1 = Parameters.red_purp 
c_control = Parameters.green
c_immunopar = Parameters.orange
c_background = CF.lighten_color(Parameters.grey_2, 0.6)
n_cols = 3
if 'FC' in model_name:
    n_cols += 1
if plot_immunoparalysis:
    n_cols += 1

# Set plot style    
CF.SetPlotParams(magnification=magnification*n_cols/4., ratio=float(2./(7.9*n_cols/4.)), fontsize=11, 
                 ylabelsize=9, xlabelsize=9, tex=True, axes_lines_w=0.5, lines_w=0.5, legendmarker=False)

# Initialize figure
fig, ax = plt.subplots(nrows=1, ncols=n_cols, sharey=True)


## Plot points
idx = 0
ax[idx].plot(X_1_0, X_2_0, ls=' ', marker='.', color=c_0, ms=ms, mew=mew, mec=mec, alpha=1.)
if 'FC' in model_name:
    if plot_test:
        ax[idx].plot(X_1_t, X_2_t, ls=' ', marker='.', color=c_control, ms=ms, mew=mew, mec=mec, alpha=1.)
ax[idx].plot(X_1_1, X_2_1, ls=' ', marker='.', color=c_1, ms=ms, mew=mew, mec=mec, alpha=1.)
ax[idx].axvline(threshold_0, color='black', alpha=1., ls='--', dashes=(1.5, 1.5), lw=1.2, 
                label='NPV: %.2f\nspe: %.2f' % (NPV_train_minNPVmodel, spe_train_minNPVmodel))
ax[idx].axvline(0, color='black', alpha=1., ls='-', lw=1.,
                label='NPV: %.2f\nspe: %.2f' % (NPV_train_model, spe_train_model)) 
ax[idx].set_xlim(x_1_min, x_1_max)
ax[idx].set_xticks(())
ax[idx].set_xlabel('x: LR score')
ax[idx].set_ylabel('y: PC1 $\perp$ x')
my_legend = ax[idx].legend(markerscale=1., scatterpoints=1, fontsize=11, handlelength=1.5, labelspacing=1.2, frameon=False, 
               loc='center right', bbox_to_anchor= (-.25, 0.5), ncol=1, borderaxespad=0)
my_legend.set_title('AUC=%.2f' % roc_train_model, prop = {'size':11}) #, 'style': 'italic'})
ax[idx].title.set_text('outcome: OTI+death')
ax[idx].grid(False)



idx = idx+1
ax[idx].plot(X_1_0, X_2_0, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
if plot_test:
    ax[idx].plot(X_1_t, X_2_t, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
ax[idx].plot(X_1_1, X_2_1, ls=' ', marker='.', color=c_1, ms=ms, mew=mew, mec=mec, alpha=0.75, label='Not survived')
ax[idx].axvline(threshold_0, color='black', alpha=1., ls='--', dashes=(1.5, 1.5), lw=1.2)
ax[idx].axvline(0, color='black', alpha=1., ls='-', lw=1.)
ax[idx].set_xlim(x_1_min, x_1_max)
ax[idx].set_xticks(())
ax[idx].set_xlabel('x: LR score')
ax[idx].title.set_text('event')
ax[idx].grid(False)



idx = idx+1
if plot_test:
    ax[idx].plot(X_1_t, X_2_t, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
ax[idx].plot(X_1_1, X_2_1, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
ax[idx].plot(X_1_0, X_2_0, ls=' ', marker='.', color=c_0, ms=ms, mew=mew, mec=mec, alpha=0.75, label='Survived')
ax[idx].axvline(threshold_0, color='black', alpha=1., ls='--', dashes=(1.5, 1.5), lw=1.2)
ax[idx].axvline(0, color='black', alpha=1., ls='-', lw=1.)
ax[idx].set_xlim(x_1_min, x_1_max)
ax[idx].set_xticks(())
ax[idx].set_xlabel('x: LR score')
ax[idx].title.set_text('no event')
ax[idx].grid(False)



if 'FC' in model_name:
    idx = idx+1
    ax[idx].plot(X_1_0, X_2_0, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
    ax[idx].plot(X_1_1, X_2_1, ls=' ', marker='.', color=c_background, ms=ms, mew=mew, mec=mec, alpha=alpha_bg)
    if plot_test:
        ax[idx].plot(X_1_t, X_2_t, ls=' ', marker='.', color=c_control, ms=ms, mew=mew, mec=mec, alpha=0.75, label='Control')
    ax[idx].axvline(threshold_0, color='black', alpha=1., ls='--', dashes=(1.5, 1.5), lw=1.2)
    ax[idx].axvline(0, color='black', alpha=1., ls='-', lw=1.)
    ax[idx].set_xlim(x_1_min, x_1_max)
    ax[idx].set_xticks(())
    ax[idx].set_xlabel('x: LR score')
    ax[idx].title.set_text('outpatients')
    ax[idx].grid(False)
    

plt.ylim(x_2_min, x_2_max)
plt.yticks(())

filename = 'ScatterplotLR#%s' % (model_name)
filename = filename + exp_description + exp_multiv_description + '.pdf'
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') #dpi=1000,

plt.show()

### Histogram

In [ ]:
magnification = 0.85
ratio = 2.4/3.7
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                        fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                        tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)

fig, ax = plt.subplots()
    
mask_class0 = y_train == 0
mask_class1 = y_train == 1
std = StandardScaler()
x = X2D_train[:,0]
x = std.fit_transform(x.reshape(-1, 1)).reshape(-1,)
x0 = x[mask_class0]
x1 = x[mask_class1]
xcontrol = X2D_test[:, 0]
xcontrol = std.transform(xcontrol.reshape(-1, 1)).reshape(-1,)
x_min = min([np.nanmin(x0), np.nanmin(x1), np.nanmin(xcontrol)])
x_max = max([np.nanmax(x0), np.nanmax(x1), np.nanmax(xcontrol)])
bins = np.linspace(x_min-0.2, x_max+0.2, 14)

color = color_0
density = False
ax.hist(x0, bins,
        alpha=0.15,
        histtype='stepfilled',
        color=color,
        density=density)
ax.hist(x0, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=density,
        facecolor='None', 
        label='no event')

bins = np.linspace(x_min-0.2, x_max+0.2, 13)

if 'FC' in model_name:
        color = color_control
        ax.hist(xcontrol, bins,
                alpha=0.2,
                histtype='stepfilled',
                color=color,
                density=density,
                label='control')
        ax.hist(xcontrol, bins,
                alpha=1.,
                histtype='step', 
                edgecolor=color,
                density=density,
                facecolor='None')

color = color_1
ax.hist(x1, bins,
        alpha=0.2,
        histtype='stepfilled',
        color=color,
        density=density,
        label='event')
ax.hist(x1, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=density,
        facecolor='None')

color_boundary = CF.lighten_color(Parameters.grey_1, 0.6)
ax.axvline(threshold_0, ls='--', lw=0.7, color='black') 

ax.title.set_text('Score Histogram')
ax.set_xlabel('%s score' % model_name)

ax.set_ylabel('n. patients')
if density:
    ax.set_ylabel('patients (density)')

if model_name=='FC+Dem':
        ax.set_yticks([0., 50., 100., 150])
        ax.set_ylim([0, 150])
if model_name=='BM+Dem':
        ax.set_ylim([0, 100])
if model_name=='CK+Dem':
       ax.set_ylim([0, 60])

leg = ax.legend(loc='center left', bbox_to_anchor=(0.98, 0.5), frameon=False)
for lh in leg.legendHandles: 
    lh.set_alpha(1)
    lh.set_edgecolor('white')

filename = 'HistogramLR#%s' % (model_name)
filename = filename + exp_description + exp_multiv_description + '.pdf'
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') #dpi=1000,
plt.show()